In [1]:
print('ok')

ok


In [2]:
%pwd

'c:\\Users\\Hamza\\Desktop\\End-to-End-Medical-Chatbot\\research'

In [3]:
import os 
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Hamza\\Desktop\\End-to-End-Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


In [6]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of text chunks ",len(text_chunks))

Length of text chunks  6970


In [3]:
def download_huggingface_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [6]:
embeddings=download_huggingface_embeddings()

In [13]:
query_result=embeddings.embed_query("Hello words")
print("length ",len(query_result))

length  384


In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc=Pinecone(api_key=PINECONE_API_KEY)

index_name='test-bot'

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)        
             

In [21]:
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [7]:
from langchain_pinecone import PineconeVectorStore

decsearch=PineconeVectorStore.from_existing_index(
    index_name='test-bot',
    embedding=embeddings
)

In [9]:
retriever=decsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [10]:
retrieved_docs=retriever.invoke('what is correlation')

In [11]:
retrieved_docs

[Document(id='3bc27224-f885-40a7-b2b9-27b94d89134f', metadata={'page': 674.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='remaining emotionally connected to the family. One\nmark of a healthy family is its capacity to allow members\nto differentiate, while family members still feel that they\nare “members in good standing” of the family.\nTriangular relationships\nFamily systems theory maintains that emotional rela-\ntionships in families are usually triangular. Whenever any\ntwo persons in the family system have problems with each\nother, they will “triangle in” a third member as a way of sta-'),
 Document(id='c3591ac3-2c27-4e8d-889b-89f48bb1c578', metadata={'page': 43.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='weak spot in the wall of an artery or heart chamber.\nCardiac shunt—A defect in the wall of the heart\nthat allows blood from different chambers to mix.\nCoronary occlusive artery disease—Blockage of\nthe

In [13]:

api_key='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'
from langchain_groq import ChatGroq
llm=ChatGroq(model_name='llama-3.3-70b-versatile',max_tokens=500,temperature=0.2,api_key=api_key)

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [15]:
system_prompt=(
    "You are an assistant for question answering task"
    "Use the following piece of retrieved context to answer"
    "the question. If you donot know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    '\n\n'
    '{context}'
)

In [16]:
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)

In [17]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [19]:
response=rag_chain.invoke({'input':'what is Chondromalacia patellae'})
print(response['answer'])

Chondromalacia patellae (CMP) is a syndrome that causes pain/discomfort at the front of the knee due to erosion of the articular cartilage of the knee joint, specifically under the kneecap (patella). It is also known as patello-femoral pain syndrome or patello-femoral stress syndrome. The condition is associated with irritation or wear on the underside of the kneecap.
